In [1]:
from lightglue import LightGlue, SuperPoint, DISK, match_pair
from lightglue.utils import load_image
import cv2
import numpy as np
import imutils
from skimage.metrics import structural_similarity as compare_ssim
from pathlib import Path

In [2]:

query_img = str(Path("bg.jpeg"))
key_img = str(Path("tape.jpeg"))

In [3]:
# SuperPoint+LightGlue
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features='superpoint', filter_confidence=0.5).eval().cuda()  # load the matcher

Loaded SuperPoint model


Loaded LightGlue model


In [4]:
image0 = load_image(Path(key_img)).cuda()
image1 = load_image(Path(query_img)).cuda()

In [5]:
feats0, feats1, matches01 = match_pair(extractor, matcher, image0, image1)


In [6]:
matches = matches01['matches']  # indices with shape (K,2) # type: ignore
points0 = feats0['keypoints'][matches[..., 0]]  # coordinates in image #0, shape (K,2) # type: ignore
points1 = feats1['keypoints'][matches[..., 1]]  # type: ignore # coordinates in image #1, shape (K,2)

In [7]:
p1 = points0.numpy()
p2 = points1.numpy()

In [8]:
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

In [9]:
img1_color = cv2.imread(key_img)
img2_color = cv2.imread(query_img) 
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

In [10]:
key_registered = cv2.warpPerspective(img1_color,
                    homography, (width, height))

In [11]:
# cv2.imwrite('key_cmp.jpg', key_registered)

In [12]:

gray_img = cv2.cvtColor(key_registered, cv2.COLOR_BGR2GRAY)
mask = (gray_img > 0).astype(np.uint8) * 255
query_masked = cv2.bitwise_and(img2_color, img2_color, mask=mask)

In [13]:
# cv2.imwrite('query_cmp.jpg', query_masked)

In [14]:


grayA = cv2.cvtColor(query_masked, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(key_registered, cv2.COLOR_BGR2GRAY)

grayA = cv2.GaussianBlur(grayA, (25,25),5.0,5.0 ,borderType=cv2.BORDER_CONSTANT)
grayB = cv2.GaussianBlur(grayB, (25,25),5.0,5.0, borderType=cv2.BORDER_CONSTANT)

# compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))

# threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 100, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)


# loop over the contours
for c in cnts:
	# compute the bounding box of the contour and then draw the
	# bounding box on both input images to represent where the two
	# images differ
	(x, y, w, h) = cv2.boundingRect(c)
	cv2.rectangle(query_masked, (x, y), (x + w, y + h), (0, 0, 255), 2)
	cv2.rectangle(key_registered, (x, y), (x + w, y + h), (0, 0, 255), 2)
# show the output images


SSIM: 0.9545295989826783


In [15]:
cv2.imshow("Original", query_masked)
cv2.imshow("Modified", key_registered)
cv2.imshow("Diff", diff)
cv2.imshow("Thresh", thresh)
while cv2.waitKey(0) != 27:
    pass
cv2.destroyAllWindows()